### 8.5.1 Confusing the eavesdropper
This example shows the different scenarios that can happen: the sender can apply a Hadamard gate, but might decide not to do that. Hence, the eavesdropper doesn't know whether to apply a Hadamard gate or not.

In [ ]:
using ImageShow
using StrangelyDisplayed
using StrangelyQuantum

The following function is the equivalent of **Listing 8.1**. All bits being sent are zeros. There are three steps before the measure step:

1. A zero or a 1 is sent
2. A Hadamard gate is either applied or not applied
3. Another Hadamard gate is either applied or not applied

This means that there are $8\equiv2^3$ combinations of steps. All combinations are demonstrated by the function:

In [ ]:
function guess()
    key_length = 8
    aliceBits = falses(key_length)

    simulator = SimpleQuantumExecutionEnvironment()
    program = Program(key_length)
    prepareStep = Step()
    superPositionStep = Step()
    superPositionStep2 = Step()
    measureStep = Step()
    for i = 1:key_length
        i > (key_length ÷ 2) && addGate(prepareStep, X(i))
        ((i - 1) ÷ 2) % 2 == 1 && addGate(superPositionStep, Hadamard(i))
        (i - 1) % 2 == 1 && addGate(superPositionStep2, Hadamard(i))
        addGate(measureStep, Measurement(i))
    end

    addStep(program, prepareStep)
    addStep(program, superPositionStep)
    addStep(program, superPositionStep2)
    addStep(program, measureStep)

    result = runProgram(simulator, program)
    qubit = getQubits(result)

    measurement = Vector{Int}(undef, key_length)
    bobBits = Vector{Bool}(undef, key_length)
    for i = 1:key_length
        measurement[i] = measure(qubit[i])
        bobBits[i] = measurement[i] == 1
        println(
            "Alice sent ",
            aliceBits[i] ? '1' : '0',
            " and Bob received ",
            bobBits[i] ? '1' : '0',
        )
    end

    return program
end

In [ ]:
program = guess()

In [ ]:
drawProgram(program)

The diagram shows that if either zero or two Hadamard gates are applied, the measurement is the same as the original qubit. If only one Hadamard gate is applied (either in step 2 or in step 3) the measurement will be wrong 50% of the time.

This means that if an eavesdropper doesn't know whether a Hadamard gate was applied to the outgoing qubit, guessing whether to apply one or not will give the wrong value 50% of the time, and will also result in incorrect data being re-transmitted to the intended recipient.